In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns
import datetime as dt

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_parquet('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip')

In [3]:
df.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2020-01-22T16:55:53.194Z,NaN,1,8.0,1998-02-07T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,4.0,0,10.0,1.0,1963-01-01T00:00:00.000Z,1997-01-11T00:00:00.000Z,NaN,NaN,NaN,0,7,0,200000.0,50000.0,1998,0,382.0,0,0.00,937.0,1,N,NaN,0,0.0,NaN,0.0,None,612.0,NaN,0.0,0,Pineapple Express - Southern,15000.0,0.0,60294.0,None,"OCEANSIDE, CITY OF",97.0,1.0,1.0,937.0,NaN,A,0,0.0,None,NaN,0,CA,Currently Unavailable,92056.0,6073.0,6.073019e+09,6.073019e+10,33.2,-117.3,23dcb0d8-3e61-45bf-899f-b951946ce2ff
1,0,2020-01-22T16:55:53.194Z,NaN,1,8.0,2005-08-29T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,NaN,1.0,1967-07-01T00:00:00.000Z,1990-07-12T00:00:00.000Z,NaN,NaN,NaN,0,7,0,100000.0,40000.0,2005,1,NaN,0,0.00,NaN,1,N,NaN,0,0.0,NaN,0.0,None,654.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,225203.0,None,NEW ORLEANS/ORLEANS PARISH*,6.0,6.0,1.0,NaN,NaN,A,0,0.0,None,NaN,0,LA,Currently Unavailable,70131.0,22071.0,2.207100e+10,2.207100e+11,29.9,-90.0,55783cdd-ccbd-4b19-930b-072def248507
2,0,2020-01-22T16:55:53.194Z,NaN,1,9.0,1998-09-28T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,10.0,1.0,1972-01-01T00:00:00.000Z,1997-07-24T00:00:00.000Z,8813.21,1720.0,0.0,0,1,0,100000.0,50000.0,1998,1,9313.0,0,8813.21,80000.0,1,N,2220.0,0,1720.0,0.0,0.0,None,133.0,NaN,0.0,0,Hurricane Georges (Panhandle),15000.0,0.0,120274.0,None,SANTA ROSA COUNTY *,NaN,NaN,1.0,100000.0,0.0,A,0,0.0,None,NaN,0,FL,Currently Unavailable,32566.0,12113.0,1.211301e+10,1.211301e+11,30.4,-86.9,bfb5922b-1b21-4882-b1d4-b3825ff53e37
3,0,2019-09-19T13:45:58.425Z,1.0,1,9.0,1994-10-07T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,10.0,1.0,1960-01-01T00:00:00.000Z,1993-10-01T00:00:00.000Z,2906.00,0.0,0.0,0,7,0,100000.0,25000.0,1994,0,4428.0,0,2906.00,100000.0,1,N,NaN,0,0.0,NaN,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,450026.0,None,"BEAUFORT, CITY OF",97.0,NaN,1.0,0.0,NaN,A,0,0.0,None,NaN,0,SC,Currently Unavailable,29902.0,45013.0,4.501300e+10,4.501300e+11,32.4,-80.7,c1cf6e00-1e6d-4493-93fc-eb430ef15495
4,0,2019-09-19T13:45:58.425Z,NaN,1,8.0,1996-03-11T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,NaN,1.0,1988-01-01T00:00:00.000Z,1996-01-11T00:00:00.000Z,3875.53,1545.0,0.0,1,7,0,100000.0,25000.0,1996,1,5252.0,0,3875.53,100000.0,1,N,3115.0,0,1545.0,0.0,0.0,None,NaN,NaN,0.0,0,None,NaN,0.0,125092.0,None,BREVARD COUNTY *,NaN,NaN,1.0,0.0,0.0,A,0,0.0,None,1.0,0,FL,Currently Unavailable,32940.0,12009.0,1.200906e+10,1.200906e+11,28.3,-80.7,cad8334c-13f2-4837-bdcf-e09591197ff7


In [113]:
# List of variables to include
df['floodEvent'].fillna('Event Name Not Recorded', inplace=True)

variables = ['floodEvent', 'buildingDamageAmount', 'buildingPropertyValue','waterDepth', 
              'floodWaterDuration', 'ratedFloodZone', 'censusTract',
              'censusBlockGroupFips', 'countyCode', 'reportedZipCode', 'rateMethod', 'causeOfDamage', 
             'totalBuildingInsuranceCoverage','occupancyType', 'originalConstructionDate', 
             'dateOfLoss', 'elevatedBuildingIndicator', 'postFIRMConstructionIndicator',
              'smallBusinessIndicatorBuilding', 'yearOfLoss', 'primaryResidenceIndicator', 'floodproofedIndicator', 
             'stateOwnedIndicator','rentalPropertyIndicator', 'state']

# Filter the DataFrame
filtered_df = df[df[variables].notna().all(axis=1)]

In [114]:
temp = filtered_df[filtered_df['waterDepth']==6]

In [115]:
# Then, count the frequency of each category
counts = temp['floodEvent'].value_counts()
# Identify the categories to keep (those with count >= 100)
to_keep = counts[counts >= 100].index
# Replace the categories to not keep with 'Other'
temp.loc[~temp['floodEvent'].isin(to_keep), 'floodEvent'] = 'Other'

temp = temp[(temp['buildingPropertyValue'] >= 10000) & (temp['buildingPropertyValue'] < 10000000)]

temp = temp[temp['floodWaterDuration'] <= 500]

temp['relativeDamage'] = temp['buildingDamageAmount']/temp['buildingPropertyValue']

temp['relativeDamage'].replace([np.inf, -np.inf], np.nan, inplace=True)

temp = temp[temp['relativeDamage'] <= 1]

zone_counts = temp['ratedFloodZone'].value_counts()
temp['ratedFloodZone'] = temp['ratedFloodZone'].apply(lambda x: x if zone_counts[x] >= 25 else 'Other')

temp.loc[temp['causeOfDamage'].isin(['9', 'D', '7']), 'causeOfDamage'] = 'other'

temp.loc[temp['occupancyType'].isin([11, 14, 12]), 'occupancyType'] = 'other'

temp = temp[temp['totalBuildingInsuranceCoverage'] <= 10000000]

In [116]:
temp['originalConstructionDate'] = pd.to_datetime(temp['originalConstructionDate'], errors='coerce')
temp['originalConstructionDate_year'] = temp['originalConstructionDate'].dt.year

temp['AgeofBuilding'] = dt.datetime.now().year - temp['originalConstructionDate_year']

temp['AgeofBuildinguntilflood'] = temp['yearOfLoss'] - temp['originalConstructionDate_year']

median_age1 = temp['AgeofBuilding'].median()
temp['AgeofBuilding'].fillna(median_age1, inplace=True)

median_age2 = temp['AgeofBuildinguntilflood'].median()
temp['AgeofBuildinguntilflood'].fillna(median_age2, inplace=True)

temp = temp[temp['AgeofBuildinguntilflood']>=0]

median_age_construction = temp['originalConstructionDate_year'].median()
temp['originalConstructionDate_year'].fillna(median_age_construction, inplace=True)

In [118]:
temp.shape[0]

34393

In [119]:
temp.to_csv('C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/filtered_csv_for_6ft.csv')

In [120]:
temp.head()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,ratedFloodZone,houseWorship,locationOfContents,lowestAdjacentGrade,lowestFloorElevation,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDamageAmount,contentsDeductibleCode,netContentsPaymentAmount,contentsPropertyValue,disasterAssistanceCoverageRequired,eventDesignationNumber,ficoNumber,floodCharacteristicsIndicator,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityNumberCurrent,nfipCommunityName,nonPaymentReasonContents,nonPaymentReasonBuilding,numberOfUnits,buildingReplacementCost,contentsReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,floodZoneCurrent,buildingDescriptionCode,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id,relativeDamage,originalConstructionDate_year,AgeofBuilding,AgeofBuildinguntilflood
52,0,2020-01-22T16:55:53.194Z,NaN,1,7.0,2004-09-16T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,2.0,0,10.0,1.0,1974-01-01 00:00:00+00:00,2000-02-15T00:00:00.000Z,112634.48,38301.23,10240.0,0,1,0,166000.0,50000.0,2004,1,189617.0,3,112634.48,200970.0,4,N,94049.0,0,38301.23,0.0,0.0,None,643.0,NaN,0.0,0,Hurricane Ivan,30000.0,10240.0,120274.0,None,SANTA ROSA COUNTY *,NaN,NaN,1.0,203000.0,0.0,R,0,6.0,None,NaN,0,FL,Currently Unavailable,32563.0,12113.0,1.211301e+10,1.211301e+11,30.4,-87.1,ef067e3d-6d8a-494a-a184-f95e3eac8958,0.943509,1974.0,49.0,30.0
60,0,2020-01-22T16:55:53.194Z,NaN,1,NaN,2005-08-28T00:00:00.000Z,0,nan,NaN,NaN,X,0,NaN,NaN,NaN,1.0,0,NaN,1.0,1979-01-01 00:00:00+00:00,2003-03-09T00:00:00.000Z,62112.62,25813.12,0.0,1,7,0,100000.0,40000.0,2005,1,62613.0,0,62112.62,124000.0,1,N,26313.0,0,25813.12,0.0,0.0,None,653.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,285256.0,None,JACKSON COUNTY*,NaN,NaN,1.0,155000.0,0.0,A,0,6.0,None,NaN,0,MS,Currently Unavailable,39563.0,28059.0,2.805904e+10,2.805904e+11,30.4,-88.5,fdbfdab5-9386-4596-b6a3-5c8adf80372f,0.504944,1979.0,44.0,26.0
524,0,2021-09-14T20:35:00.880Z,NaN,1,8.0,2005-08-29T00:00:00.000Z,0,nan,6.0,13.0,A14,0,NaN,NaN,19.0,1.0,0,NaN,1.0,1979-01-01 00:00:00+00:00,1993-08-15T00:00:00.000Z,185900.00,42200.00,0.0,1,1,0,185900.0,42200.0,2005,1,186400.0,0,185900.00,186400.0,1,N,42700.0,0,42200.00,0.0,0.0,None,653.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,285257.0,285257,"LONG BEACH, CITY OF",NaN,NaN,1.0,195900.0,0.0,R,0,6.0,X,1.0,0,MS,Currently Unavailable,39560.0,28047.0,2.804700e+10,2.804700e+11,30.4,-89.1,18430e1d-620e-4158-bbd9-e1d6143c8c4e,1.000000,1979.0,44.0,26.0
531,0,2020-01-22T16:55:53.194Z,NaN,1,8.0,2005-08-29T00:00:00.000Z,0,nan,NaN,NaN,AE,0,NaN,NaN,NaN,1.0,0,NaN,1.0,1971-01-01 00:00:00+00:00,2002-09-12T00:00:00.000Z,73300.00,5400.00,0.0,0,1,0,73300.0,5400.0,2005,1,152725.0,1,73300.00,182928.0,1,N,23100.0,1,5400.00,0.0,0.0,None,654.0,NaN,0.0,0,Hurricane Katrina,30000.0,0.0,225203.0,None,NEW ORLEANS/ORLEANS PARISH*,NaN,NaN,1.0,184776.0,0.0,A,0,6.0,None,NaN,0,LA,Currently Unavailable,70122.0,22071.0,2.207100e+10,2.207100e+11,30.0,-90.1,29686683-7132-4cf2-85a9-1bb83854a8fe,0.834891,1971.0,52.0,34.0
649,0,2020-01-22T16:55:53.194Z,NaN,1,7.0,2017-08-27T00:00:00.000Z,0,nan,NaN,NaN,AE,0,NaN,100.0,NaN,2.0,0,NaN,1.0,1972-01-01 00:00:00+00:00,2001-08-09T00:00:00.000Z,231558.65,71900.00,0.0,0,B,0,250000.0,71900.0,2017,1,210939.0,2,231558.

In [51]:
df_test = df[df['censusBlockGroupFips'].astype(str).str.contains('1208797')]

In [52]:
df_test.shape[0]

26487

In [53]:
df_test['censusBlockGroupFips'].unique()

array([1.20879712e+11, 1.20879708e+11, 1.20879719e+11, 1.20879710e+11,
       1.20879711e+11, 1.20879714e+11, 1.20879720e+11, 1.20879709e+11,
       1.20879718e+11, 1.20879714e+11, 1.20879715e+11, 1.20879704e+11,
       1.20879712e+11, 1.20879711e+11, 1.20879705e+11, 1.20879715e+11,
       1.20879702e+11, 1.20879704e+11, 1.20879717e+11, 1.20879716e+11,
       1.20879710e+11, 1.20879723e+11, 1.20879720e+11, 1.20879714e+11,
       1.20879724e+11, 1.20879713e+11, 1.20879707e+11, 1.20879721e+11,
       1.20879720e+11, 1.20879715e+11, 1.20879710e+11, 1.20879708e+11,
       1.20879719e+11, 1.20879703e+11, 1.20879715e+11, 1.20879714e+11,
       1.20879722e+11, 1.20879706e+11, 1.20879706e+11, 1.20879719e+11,
       1.20879708e+11, 1.20879710e+11, 1.20879709e+11, 1.20879707e+11,
       1.20879704e+11, 1.20879705e+11, 1.20879708e+11, 1.20879707e+11,
       1.20879726e+11, 1.20879715e+11, 1.20879718e+11, 1.20879722e+11,
       1.20879722e+11, 1.20879704e+11, 1.20879724e+11, 1.20879713e+11,
      

In [16]:
df_kat = df[df['floodEvent']=='Hurricane Dennis']

In [17]:
df_kat.shape[0]

8230

In [15]:
df_kat[df['state']=='AL']['dateOfLoss'].unique()

C:\Users\Asus\AppData\Local\Temp\ipykernel_30116\3657085137.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_kat[df['state']=='AL']['dateOfLoss'].unique()


array(['2005-08-29T00:00:00.000Z', '2005-08-30T00:00:00.000Z',
       '2005-08-28T00:00:00.000Z'], dtype=object)

In [19]:
df_kat['state'].unique()

array(['AL', 'NC', 'FL', 'GA', 'SC', 'MS'], dtype=object)

In [46]:
df_new = df[(df['buildingPropertyValue'] >= 1000) & (df['buildingPropertyValue'] < 10000000)]
df_new = df_new[~df_new['waterDepth'].isna()]

df_new['relativeDamage'] = df_new['buildingDamageAmount']/df_new['buildingPropertyValue']

df_new['relativeDamage'].replace([np.inf, -np.inf], np.nan, inplace=True)

df_new = df_new[df_new['relativeDamage'] <= 1]

In [47]:
df_new[df_new['waterDepth']==1]['relativeDamage'].mean()

0.19859928406207078